Note: This testing notebook walks through the use of the manual and NLP feature extraction techniques and runs those features through 5 sets of MLP neural network tests. The  two manual feature related functions and three NLP feature related functions are called from the respective .py files in the codebase.

Note: any code used from a source has been reference via a comment directly next to that specific line of code. All coding references used for the manual and NLP feature extraction techniques can be found directly in their respective .py files in the codebase.

# Table of Contents

1. Import Sample Dataset

1. Run Files Containing Functions

1. Sorting Data with SQL for Batched Features

# Import Sample Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

In [3]:
data_df = pd.read_csv('/content/drive/My Drive/ECE 697 Project/10. Final Presentation/codebase sample dataset/codebase_sample_dataset_SUEE1_98900_to_99399.csv')
labels = pd.read_csv('/content/drive/My Drive/ECE 697 Project/10. Final Presentation/codebase sample dataset/codebase_sample_dataset_SUEE1_98900_to_99399_labels.csv', header = None)
data_df.columns =['No','Time','Source','Destination','Protocol','Length','Info']
labels.columns = ['label']

In [4]:
data_df

,No,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.168.1.6,192.168.0.1,TCP,54,52531 > 80 [ACK] Seq=1 Ack=1 Win=253 Len=0
1,2,0.011750,192.168.1.6,192.168.0.1,TCP,54,"52531 > 80 [FIN, ACK] Seq=1 Ack=1 Win=253 Len=0"
2,3,0.011760,192.168.0.1,192.168.1.6,TCP,54,80 > 52531 [ACK] Seq=1 Ack=2 Win=237 Len=0
3,4,0.325331,192.168.0.8,192.168.0.1,TCP,74,54236 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
4,5,0.325364,192.168.0.1,192.168.0.8,TCP,74,"80 > 54236 [SYN, ACK] Seq=0 Ack=1 Win=28960 ..."
...,...,...,...,...,...,...,...
495,496,1454.171676,10.128.0.88,192.168.1.9,TCP,54,50948 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0
496,497,1454.171685,10.128.0.82,192.168.1.9,TCP,54,44318 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0
497,498,1454.171686,10.128.0.88,192.168.1.9,TCP,54,33222 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0
498,499,1454.171691,10.128.0.82,192.168.1.9,TCP,54,51034 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0


In [5]:
labels

,label
0,0
1,0
2,0
3,0
4,0
...,...
495,1
496,1
497,1
498,1


# Run Files Containing Functions

In [6]:
  !pip install torch-summary 
  !pip install datasets 
  !pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
#File containing manual feature extraction function and batched manual feature extraction function
%run '/content/drive/My Drive/ECE 697 Project/8. Classification Testing/NN and Clustering using Manual FE/Final Evaluation/manual_feature_data_generation_functions.py'

#File containing NLP fine tuning function, NLP feature extraction function, and batched NLP feature extraction function
%run '/content/drive/My Drive/ECE 697 Project/8. Classification Testing/NN and Clustering using Manual FE/Final Evaluation/nlp_feature_data_generation_functions.py'

# Sorting Data via SQL for Batched Features

In [8]:
data_with_labels = pd.concat([data_df,labels],axis = 1)

In [9]:
np.shape(data_with_labels)

(500, 8)

In [17]:
connection_to_db = create_engine('sqlite:///sample_database.db')  #https://uwmadison.app.box.com/s/05pdwm1sebn77ge4se8od25q7ws79k1z
data_with_labels.to_sql('sample_data', con=connection_to_db, if_exists='replace')  #https://uwmadison.app.box.com/s/05pdwm1sebn77ge4se8od25q7ws79k1z

In [18]:
#https://uwmadison.app.box.com/s/05pdwm1sebn77ge4se8od25q7ws79k1z
%load_ext sql
%sql sqlite:///sample_database.db #

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
(sqlite3.OperationalError) unrecognized token: "#"
[SQL: #]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [19]:
#https://uwmadison.app.box.com/s/05pdwm1sebn77ge4se8od25q7ws79k1z
%%sql

DROP TABLE IF EXISTS data_grouped;

CREATE TABLE data_grouped AS 
SELECT No,Time, Source, Destination, Protocol, Length, Info, label FROM sample_data ORDER BY Source ASC

 * sqlite:///sample_database.db
Done.
Done.


[]

In [21]:
result = %sql SELECT * FROM data_grouped
data_df_grouped = result.DataFrame()

 * sqlite:///sample_database.db
Done.


In [22]:
data_df_grouped

,No,Time,Source,Destination,Protocol,Length,Info,label
0,248,1454.124546,10.128.0.50,192.168.1.9,TCP,74,54704 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...,1
1,255,1454.124587,10.128.0.50,192.168.1.9,TCP,74,48260 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...,1
2,257,1454.124607,10.128.0.50,192.168.1.9,TCP,54,54704 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0,1
3,258,1454.124665,10.128.0.50,192.168.1.9,HTTP,254,GET /bcrypt.php HTTP/1.1,1
4,260,1454.124669,10.128.0.50,192.168.1.9,TCP,54,48260 > 80 [ACK] Seq=1 Ack=1 Win=29312 Len=0,1
...,...,...,...,...,...,...,...,...
495,485,1454.168024,192.168.1.9,10.128.0.57,TCP,66,"80 > 36659 [SYN, ACK] Seq=0 Ack=1 Win=42340 ...",0
496,492,1454.171596,192.168.1.9,10.128.0.88,TCP,66,"80 > 50948 [SYN, ACK] Seq=0 Ack=1 Win=42340 ...",0
497,493,1454.171596,192.168.1.9,10.128.0.88,TCP,66,"80 > 33222 [SYN, ACK] Seq=0 Ack=1 Win=42340 ...",0
498,494,1454.171610,192.168.1.9,10.128.0.82,TCP,66,"80 > 44318 [SYN, ACK] Seq=0 Ack=1 Win=42340 ...",0


# Generating Manual Features

In [23]:
manual_features = MF(data_df)

In [24]:
np.shape(manual_features)

(500, 16)

# Generating Batched Manual Features

In [26]:
manual_features_sorted = MF(data_df_grouped)
manual_features_sorted = pd.DataFrame(manual_features_sorted)
manual_features_sorted = pd.concat([data_df_grouped,manual_features_sorted],axis = 1)

manual_features_batched = MF_batching(manual_features_sorted,10,500)

In [27]:
np.shape(manual_features_batched)

(75, 23)

In [38]:
manual_features_batched[:,22]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

# Fine Tune Distilbert NLP

In [31]:
FTmodel = FT_NLP(data_df,labels)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




# Generate NLP Features

In [32]:
NLP_features = NLP_Features(data_df,labels,FTmodel)

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf

(150, 768)
(300, 768)
(450, 768)
(500, 768)


In [33]:
np.shape(NLP_features)

(500, 768)

# Generate Batched NLP Features

In [34]:
NLP_features = pd.DataFrame(NLP_features)
NLP_features_for_batching = pd.concat([data_df,NLP_features,labels], axis = 1)
NLP_features_for_batching=NLP_features_for_batching.drop(columns=['No'])
NLP_batched_features = NLP_batching(NLP_features_for_batching,10,500)
np.shape(NLP_batched_features)

(318, 769)

In [39]:
NLP_batched_features

array([[-0.70106429, -0.05377735, -0.19084324, ...,  0.21668667,
         0.63650763,  0.        ],
       [-0.84879917, -0.2362639 , -0.26129329, ...,  0.26905975,
         0.87255764,  0.        ],
       [-0.81481814, -0.0699446 , -0.17096676, ...,  0.35730788,
         0.8593387 ,  0.        ],
       ...,
       [-0.57059044, -0.1228254 , -0.06960668, ...,  0.27302089,
         0.71632355,  1.        ],
       [-0.59152359, -0.07591073, -0.12683605, ...,  0.30311528,
         0.79950064,  1.        ],
       [-0.5593341 , -0.04973866, -0.02431287, ...,  0.25047553,
         0.65828454,  1.        ]])

# Consolidate All Features and Labels for Testing

In [55]:
feature_list = [manual_features,manual_features_batched[:,0:22],NLP_features,NLP_batched_features[:,0:768]]
label_list = [labels,manual_features_batched[:,22],labels,NLP_batched_features[:,768]]
feature_names = ['manual features', 'batched manual features', 'NLP features', 'batched NLP features']
input_count = [16,22,768,768]
dense_count = [12,18,576,576]

# Test 1: MLP w/o dropout

In [50]:
for i in range(0,4):

  X_train, X_test, y_train, y_test = train_test_split(feature_list[i], label_list[i], test_size=0.3, random_state=42)

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  X_train = X_train.astype(np.float64)
  X_test = X_test.astype(np.float64)

  model = Sequential()
  model.add(Dense(dense_count[i], input_dim=input_count[i], activation='relu'))
  model.add(Dense(dense_count[i], activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train.astype(np.float64), epochs=20, batch_size=10)

  pred = model.predict(X_test) 
  y_pred = np.rint(pred)
  print(feature_names[i], ' confusion matrix results: ')
  print(confusion_matrix(y_test.astype(np.float64), y_pred))
  print()

Epoch 1/20
35/35 [==============================] - 1s 3ms/step - loss: 0.6909 - accuracy: 0.5114
Epoch 2/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6625 - accuracy: 0.5743
Epoch 3/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6427 - accuracy: 0.6514
Epoch 4/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6269 - accuracy: 0.7029
Epoch 5/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6126 - accuracy: 0.7314
Epoch 6/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5989 - accuracy: 0.7343
Epoch 7/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5861 - accuracy: 0.7314
Epoch 8/20
35/35 [==============================] - 0s 2ms/step - loss: 0.5743 - accuracy: 0.7286
Epoch 9/20
35/35 [==============================] - 0s 2ms/step - loss: 0.5620 - accuracy: 0.7600
Epoch 10/20
35/35 [==============================] - 0s 2ms/step - loss: 0.5510 - accuracy: 0.7571
Epoch 11/20
35/35 [

# Test 2: MLP w/ dropout

In [53]:
for i in range(0,4):

  X_train, X_test, y_train, y_test = train_test_split(feature_list[i], label_list[i], test_size=0.3, random_state=42)

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  X_train = X_train.astype(np.float64)
  X_test = X_test.astype(np.float64)

  model = Sequential()
  model.add(Dense(dense_count[i], input_dim=input_count[i], activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(dense_count[i], activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(dense_count[i], activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train.astype(np.float64), epochs=20, batch_size=10)

  pred = model.predict(X_test) 
  y_pred = np.rint(pred)
  print(feature_names[i], ' confusion matrix results: ')
  print(confusion_matrix(y_test.astype(np.float64), y_pred))
  print()


Epoch 1/20
35/35 [==============================] - 1s 2ms/step - loss: 0.7378 - accuracy: 0.4914
Epoch 2/20
35/35 [==============================] - 0s 2ms/step - loss: 0.7058 - accuracy: 0.5457
Epoch 3/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.5857
Epoch 4/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6748 - accuracy: 0.6257
Epoch 5/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6656 - accuracy: 0.5971
Epoch 6/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.6400
Epoch 7/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6483 - accuracy: 0.6171
Epoch 8/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6325 - accuracy: 0.6429
Epoch 9/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.6686
Epoch 10/20
35/35 [==============================] - 0s 2ms/step - loss: 0.6073 - accuracy: 0.6886
Epoch 11/20
35/35 [

# Test 3: K Means

In [54]:
for i in range(0,4):

  X_train, X_test, y_train, y_test = train_test_split(feature_list[i], label_list[i], test_size=0.3, random_state=42)

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  X_train = X_train.astype(np.float64)
  X_test = X_test.astype(np.float64)

  kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)
  cluster_pred = kmeans.predict(X_test)
  print(feature_names[i], ' confusion matrix results: ')
  print(confusion_matrix(y_test.astype(np.float64), cluster_pred))
  print()

manual features  confusion matrix results: 
[[13 61]
 [12 64]]

batched manual features  confusion matrix results: 
[[ 1  6]
 [ 0 16]]

NLP features  confusion matrix results: 
[[10 64]
 [76  0]]

batched NLP features  confusion matrix results: 
[[ 4 45]
 [47  0]]



# Test 4: MLP and K Means Averaging

In [57]:
for i in range(0,4):

  X_train, X_test, y_train, y_test = train_test_split(feature_list[i], label_list[i], test_size=0.3, random_state=42)

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  X_train = X_train.astype(np.float64)
  X_test = X_test.astype(np.float64)

  model = Sequential()
  model.add(Dense(dense_count[i], input_dim=input_count[i], activation='relu'))
  model.add(Dense(dense_count[i], activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train.astype(np.float64), epochs=20, batch_size=10)

  pred = model.predict(X_test) 
  
  kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)
  cluster_pred = kmeans.predict(X_test)

  a,b = np.shape(X_test)
  
  for j in range(0,a):
    pred[j] = (pred[j]+cluster_pred[j])/2

  y_pred = np.rint(pred)

  print(feature_names[i], ' confusion matrix results: ')
  print(confusion_matrix(y_test.astype(np.float64), y_pred))
  print()

Epoch 1/20
35/35 [==============================] - 1s 3ms/step - loss: 0.7129 - accuracy: 0.6286
Epoch 2/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6593 - accuracy: 0.6371
Epoch 3/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6209 - accuracy: 0.7514
Epoch 4/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5981 - accuracy: 0.7457
Epoch 5/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5808 - accuracy: 0.7486
Epoch 6/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5664 - accuracy: 0.7457
Epoch 7/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5528 - accuracy: 0.7571
Epoch 8/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5398 - accuracy: 0.7571
Epoch 9/20
35/35 [==============================] - 0s 4ms/step - loss: 0.5266 - accuracy: 0.7571
Epoch 10/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5144 - accuracy: 0.7571
Epoch 11/20
35/35 [

# Test 5: MLP using K Means 

In [58]:
for i in range(0,4):

  X_train, X_test, y_train, y_test = train_test_split(feature_list[i], label_list[i], test_size=0.3, random_state=42)

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)

  X_train = X_train.astype(np.float64)
  X_test = X_test.astype(np.float64)

  kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)
  cluster_pred = kmeans.predict(X_train)
  cluster_pred = cluster_pred.reshape(-1,1)
  X_train = np.concatenate((X_train,cluster_pred),axis=1)

  cluster_pred = kmeans.predict(X_test)
  cluster_pred = cluster_pred.reshape(-1,1)
  X_test = np.concatenate((X_test,cluster_pred),axis=1)

  model = Sequential()
  model.add(Dense(dense_count[i], input_dim=input_count[i]+1, activation='relu'))
  model.add(Dense(dense_count[i], activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train.astype(np.float64), epochs=20, batch_size=10)

  pred = model.predict(X_test) 
  y_pred = np.rint(pred)

  print(feature_names[i], ' confusion matrix results: ')
  print(confusion_matrix(y_test.astype(np.float64), y_pred))
  print()

Epoch 1/20
35/35 [==============================] - 1s 3ms/step - loss: 0.6858 - accuracy: 0.6057
Epoch 2/20
35/35 [==============================] - 0s 3ms/step - loss: 0.6384 - accuracy: 0.6171
Epoch 3/20
35/35 [==============================] - 0s 4ms/step - loss: 0.6130 - accuracy: 0.6143
Epoch 4/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5963 - accuracy: 0.6200
Epoch 5/20
35/35 [==============================] - 0s 4ms/step - loss: 0.5803 - accuracy: 0.6543
Epoch 6/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5679 - accuracy: 0.7371
Epoch 7/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5563 - accuracy: 0.7514
Epoch 8/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5452 - accuracy: 0.7486
Epoch 9/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5342 - accuracy: 0.7657
Epoch 10/20
35/35 [==============================] - 0s 3ms/step - loss: 0.5243 - accuracy: 0.7514
Epoch 11/20
35/35 [